# Bluesky & ophyd: Lesson 1 - EpicsScaler

## Starting this session in a Jupyter notebook

This session was started from the linux command line:

```
jemian@otz .../JEMIAN/Documents $ /APSshare/anaconda3/BlueSky/bin/jupyter-notebook 
```

This command produced the following console output and then started my default web browser with a one-time-token-authenticated connection to the Jupyter Notebook server (still running in the console):

```
[I 15:16:57.546 NotebookApp] Serving notebooks from local directory: /home/oxygen18/JEMIAN/Documents
[I 15:16:57.546 NotebookApp] 0 active kernels
[I 15:16:57.546 NotebookApp] The Jupyter Notebook is running at:
[I 15:16:57.546 NotebookApp] http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:16:57.546 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 15:16:57.547 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:17:00.863 NotebookApp] Accepting one-time-token-authenticated connection from ::1
```

Next, found the **New** drop-down menu button (top right, below the **Lougout** button) and selected *Python 3* to start a new notebook page using a Python 3 shell (the only kind available here).

Finally, from the **File** menu in the jupyter notebook (in the browser), selected *Rename ...* to save the *Untitled* notebook with the name *lesson1* (default extension is `.ipynb`).

## Connect an EPICS scaler

I have a synApps (v5.8) *XXX*-style IOC with the prefix `gov:`.  It has a [scaler](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/std/R3-4-1/documentation/scalerRecord.html), 16 soft channel [motor](https://github.com/epics-modules/motor)s, and some other support we'll ignore in lesson 1.

The scaler is `gov:scaler1`.  We'll connect to that first.  To make the connection, we need to import the [`ScalerCH`](http://nsls-ii.github.io/ophyd/builtin-devices.html#epicsscaler) device from the [`ophyd.scaler`](http://nsls-ii.github.io/ophyd/) library.

In [1]:
from ophyd.scaler import ScalerCH

Now we can create the scaler object we'll use as a detector.

In [21]:
scaler = ScalerCH("gov:scaler1", name="scaler")

Let's test that connection by asking the scaler to read its values from EPICS.

In [22]:
scaler.read()

OrderedDict([('clock', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I0', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('diode', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('scaler_time', {'value': 0.0, 'timestamp': 1532469891.09066})])

By default, the ophyd `ScalerCH` object shows data for only the channels with names defined in EPICS.  Except for one channel (probably a bug in the `ScalerCH` code).  To fix that, we'll load another support library.  (Loading additional support as we need it will be a common theme in these lessons.  Rather than loading all the libraries first, as is common in python code files, we'll load support code as the need arises.)

The new support code is `use_EPICS_scaler_channels` from [`APS_BlueSky_tools.devices`](http://aps-bluesky-tools.readthedocs.io/en/latest/source_code/devices.html#APS_BlueSky_tools.devices.use_EPICS_scaler_channels).

Our scaler only has a few channels in use (that is, channels where the name of the channel has been specified in the GUI screen).  Let's focus on just those channels.

To read just the *named* channels, we'll load an additional support library.  (Loading additional support as we need it will be a common theme in these lessons.  Rather than loading all the libraries first, as is common in python code files, we'll load support code as the need arises.)

The new support code is `use_EPICS_scaler_channels` from [`APS_BlueSky_tools.devices`](http://aps-bluesky-tools.readthedocs.io/en/latest/source_code/devices.html#APS_BlueSky_tools.devices.use_EPICS_scaler_channels).

In [23]:
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels

That is a utility function that will modify our `scaler` so it only reports the named channels (edits out the one empty named channel).  Now the report is succinct.

In [24]:
scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('clock', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I0', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('diode', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('scaler_time', {'value': 0.0, 'timestamp': 1532469891.09066})])

If you want to change the name fields on any of the scaler channels from the command line, follow this example.

In [25]:
scaler.channels.chan04.chname.put("scint")
scaler.channels.chan07.chname.put("roi1")

Then, update the `scaler` object for these channels.

In [26]:
scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('clock', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I0', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('I', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('scint', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('roi1', {'value': 0.0, 'timestamp': 1532469891.09066}),
             ('scaler_time', {'value': 0.0, 'timestamp': 1532469891.09066})])

## Get, then Set the count time on the scaler

The preset counting time (`.TP`) is one of the many fields of the EPICS scaler record.  It is available as `scaler.preset_time` in this scaler object.  We can inspect this:

In [27]:
scaler.preset_time

EpicsSignal(read_pv='gov:scaler1.TP', name='scaler_preset_time', parent='scaler', value=1.5, timestamp=1532469891.09066, tolerance=0.001, pv_kw={}, auto_monitor=False, string=False, write_pv='gov:scaler1.TP', limits=False, put_complete=False)

Since `scaler.preset_time` is an [`EpicsSignal`](https://github.com/NSLS-II/ophyd/blob/master/ophyd/signal.py#L647), ([Signal](http://nsls-ii.github.io/ophyd/signals.html) is a fundamental ophyd structure), we print its `.value`.

In [30]:
scaler.preset_time.value

3.0

Set the scaler to count for 1.5 seconds once it is told to count.  We'll count it next.

In [31]:
scaler.preset_time.put(1.5)

Note, to tell the scaler to count, we put a 1 to its `.CNT` field.

In [32]:
scaler.count.put(1)

Now, read the scaler again.  This scaler is not very interesting since it is not connected to any hardware.  But, the timestamps have changed.

In [33]:
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1532551575.410314}),
             ('I0', {'value': 6.0, 'timestamp': 1532551575.410314}),
             ('I', {'value': 7.0, 'timestamp': 1532551575.410314}),
             ('scint', {'value': 7.0, 'timestamp': 1532551575.410314}),
             ('roi1', {'value': 8.0, 'timestamp': 1532551575.410314}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1532551575.410314})])

## Use Bluesky to count the scaler

So far, we have used ophyd to talk with the EPICS scaler.  Let's start to use Bluesky.  We need to load support to start the Bluesky [*RunEngine*](http://nsls-ii.github.io/bluesky/tutorial.html#the-runengine).  For now, we'll use the most basic configuration (does not save data anywhere).  

The job of the RunEngine is to process command messages (our data acquisition commands) and to output documents (the data to be acquired).  Initially, we'll use predefined data acquisition sequences (called [*plans*](http://nsls-ii.github.io/bluesky/plans.html)) and ignore the [documents](http://nsls-ii.github.io/bluesky/documents.html).

In [34]:
from bluesky import RunEngine
RE = RunEngine({})

Next, we'll load a library with many [predefined plans](http://nsls-ii.github.io/bluesky/plans.html#summary).  The first plan we want to use is [`count()`](http://nsls-ii.github.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  By the way, we'll use a python feature to rename `bluesky.plans` to the shorter `bp` since it will be used a lot.

In [35]:
import bluesky.plans as bp

Now we can count the `scaler` by submitting the `bp.count` plan to the RunEngine instance `RE`.  The `bp.count()` plan takes one argument, a *list* of countable objects (just in case you want to count more than one detector at the same time).  Our list has only one object: `scaler`

In [36]:
RE(bp.count([scaler]))

('13e9397b-04c0-4a7e-892a-ce01ac1a17d1',)

The final result from submitting a plan to the RunEngine is an identifier of the sequence of documents.  Since we did not capture that sequence, we can't view it or access it in any way.  Let's *fix* that next.

### Report about the documents emitted from the RunEngine

To display information about the documents emitted from the RunEngine, we need to make a function that will receive the documents.  

In Bluesky terms, this type of function is a *callback*.  It takes two arguments.  The first argument is a `str` that tells what kind of document is coming, the second is a python dictionary with the document's contents.  We'll start by printing summary information.  We then submit the plan again with the name of our callback function as a second argument (not to `bp.count` but as a second argument to `RE()`).

In [37]:
def myCallback(key, doc):
    print(key, len(doc))

In [38]:
RE(bp.count([scaler]), myCallback)

start 10
descriptor 8
event 7
stop 6


('9e3bf71a-919b-40ad-854b-24eec20b7a4c',)

So, there were four documents.  See http://nsls-ii.github.io/bluesky/documents.html for the details of each.  Let's extend our callback to print the details of each.

In [39]:
def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")

In [40]:
RE(bp.count([scaler]), myCallback)

start 10
	 uid 9bd9a9d1-99ee-4338-b29a-12e5bfb51994
	 time 1532551634.5537052
	 scan_id 3
	 hints {'dimensions': [(('time',), 'primary')]}
	 plan_name count
	 plan_type generator
	 detectors ['scaler']
	 num_points 1
	 plan_args {'detectors': ["ScalerCH(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'channels.chan04', 'channels.chan04.s', 'channels.chan07', 'channels.chan07.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan03', 'channels.chan03.chname', 'channels.chan03.preset', 'channels.chan03.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 'channels.chan05.chname', 'channels.chan05.p

('9bd9a9d1-99ee-4338-b29a-12e5bfb51994',)

With the details, we see more of what is happening.  

The first document is a `start` and it says what is happening and how it will be identified.  The identifier (named `uid`) is a [UUID](https://docs.python.org/3.6/library/uuid.html) (specifically, a [uuid4](https://docs.python.org/3.6/library/uuid.html#uuid.uuid4)).  This looks tedious and random.  The one good thing is that these are unique and often can be referred to by the first few characters (6-8 are enough to be *probably* unique).  The `uid` from the `start` document is what the RunEngine reports when the scan stops.

The second document is a `descriptor` and it says what, exactly, will be measured.  It also includes a reference to `start`'s `uid`.  We can follow the chain back to `start` this way.

The third document is an `event` and it provides the data reading when the scaler was counted.  The data is read according to the `descriptor` document (just actual values for the *named* channels).

The last document is a `stop` document that describes how things ended.  It also says there was only one stream of event documents named *primary*.

Let's repeat that plan, this time asking to `count` the scaler three times.  This is the `num=3` addition to the call to `bp.count()`.  What's tricky now is keeping track of the parentheses and square braces.

In [41]:
RE(bp.count([scaler], num=3), myCallback)

start 10
	 uid 600c332e-39a3-4476-925d-b3df8bc87a6a
	 time 1532551732.896808
	 scan_id 4
	 hints {'dimensions': [(('time',), 'primary')]}
	 plan_name count
	 plan_type generator
	 detectors ['scaler']
	 num_points 3
	 plan_args {'detectors': ["ScalerCH(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'channels.chan04', 'channels.chan04.s', 'channels.chan07', 'channels.chan07.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan03', 'channels.chan03.chname', 'channels.chan03.preset', 'channels.chan03.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 'channels.chan05.chname', 'channels.chan05.pr

event 7
	 descriptor 9bc37313-3596-4319-9400-e9941c20bf64
	 time 1532551736.538063
	 data {'clock': 16000000.0, 'I0': 7.0, 'I': 8.0, 'scint': 8.0, 'roi1': 8.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1532551736.532659, 'I0': 1532551736.532659, 'I': 1532551736.532659, 'scint': 1532551736.532659, 'roi1': 1532551736.532659, 'scaler_time': 1532551736.532659}
	 seq_num 2
	 uid 52881a26-aa3b-48b1-8535-76772212f770
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor 9bc37313-3596-4319-9400-e9941c20bf64
	 time 1532551738.166475
	 data {'clock': 16000000.0, 'I0': 5.0, 'I': 6.0, 'scint': 7.0, 'roi1': 7.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1532551738.161586, 'I0': 1532551738.161586, 'I': 1532551738.161586, 'scint': 1532551738.161586, 'roi1': 1532551738.161586, 'scaler_time': 1532551738.161586}
	 seq_num 3
	 uid d3a80b5a-d641-4ee7-ac45-18a912241273
	 filled {}
~~~~~~~~~~~~~~~~~
stop 6
	 run_start 600c332e-39a3-4476-925d-b3df8bc87a6a
	 time 1532551738.1710577
	 uid 423cdc95-e449-4894-8d9

('600c332e-39a3-4476-925d-b3df8bc87a6a',)

Mostly the same but now there are three `event` documents.

# Summary

We'll show the code so far as a proper python program now:

```
#!/APSshare/anaconda3/BlueSky/bin/python

"lesson 1: EpicsScaler"

from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")


RE = RunEngine({})

scaler = ScalerCH("gov:scaler1", name="scaler")
scaler.preset_time.put(1.5)
print(scaler.preset_time.value)

scaler.channels.chan04.chname.put("scint")
scaler.channels.chan07.chname.put("roi1")

scaler.match_names()
use_EPICS_scaler_channels(scaler)
print(scaler.read())
print(RE(bp.count([scaler], num=3), myCallback))
```


Here is the output when run from the console:

```
jemian@otz .../use_bluesky/notebooks $ ./lesson1.py 
1.5
OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1532551738.161586}), ('I0', {'value': 5.0, 'timestamp': 1532551738.161586}), ('I', {'value': 6.0, 'timestamp': 1532551738.161586}), ('scint', {'value': 7.0, 'timestamp': 1532551738.161586}), ('roi1', {'value': 7.0, 'timestamp': 1532551738.161586}), ('scaler_time', {'value': 1.6, 'timestamp': 1532551738.161586})])
start 10
	 uid e2874f6b-7ea2-47eb-a000-80889341b212
	 time 1532552744.9009314
	 plan_args {'detectors': ["ScalerCH(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'channels.chan04', 'channels.chan04.s', 'channels.chan07', 'channels.chan07.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan03', 'channels.chan03.chname', 'channels.chan03.preset', 'channels.chan03.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 'channels.chan05.chname', 'channels.chan05.preset', 'channels.chan05.gate', 'channels.chan06', 'channels.chan06.chname', 'channels.chan06.preset', 'channels.chan06.gate', 'channels.chan07', 'channels.chan07.chname', 'channels.chan07.preset', 'channels.chan07.gate', 'channels.chan08', 'channels.chan08.chname', 'channels.chan08.preset', 'channels.chan08.gate', 'channels.chan09', 'channels.chan09.chname', 'channels.chan09.preset', 'channels.chan09.gate', 'channels.chan10', 'channels.chan10.chname', 'channels.chan10.preset', 'channels.chan10.gate', 'channels.chan11', 'channels.chan11.chname', 'channels.chan11.preset', 'channels.chan11.gate', 'channels.chan12', 'channels.chan12.chname', 'channels.chan12.preset', 'channels.chan12.gate', 'channels.chan13', 'channels.chan13.chname', 'channels.chan13.preset', 'channels.chan13.gate', 'channels.chan14', 'channels.chan14.chname', 'channels.chan14.preset', 'channels.chan14.gate', 'channels.chan15', 'channels.chan15.chname', 'channels.chan15.preset', 'channels.chan15.gate', 'channels.chan16', 'channels.chan16.chname', 'channels.chan16.preset', 'channels.chan16.gate', 'channels.chan17', 'channels.chan17.chname', 'channels.chan17.preset', 'channels.chan17.gate', 'channels.chan18', 'channels.chan18.chname', 'channels.chan18.preset', 'channels.chan18.gate', 'channels.chan19', 'channels.chan19.chname', 'channels.chan19.preset', 'channels.chan19.gate', 'channels.chan20', 'channels.chan20.chname', 'channels.chan20.preset', 'channels.chan20.gate', 'channels.chan21', 'channels.chan21.chname', 'channels.chan21.preset', 'channels.chan21.gate', 'channels.chan22', 'channels.chan22.chname', 'channels.chan22.preset', 'channels.chan22.gate', 'channels.chan23', 'channels.chan23.chname', 'channels.chan23.preset', 'channels.chan23.gate', 'channels.chan24', 'channels.chan24.chname', 'channels.chan24.preset', 'channels.chan24.gate', 'channels.chan25', 'channels.chan25.chname', 'channels.chan25.preset', 'channels.chan25.gate', 'channels.chan26', 'channels.chan26.chname', 'channels.chan26.preset', 'channels.chan26.gate', 'channels.chan27', 'channels.chan27.chname', 'channels.chan27.preset', 'channels.chan27.gate', 'channels.chan28', 'channels.chan28.chname', 'channels.chan28.preset', 'channels.chan28.gate', 'channels.chan29', 'channels.chan29.chname', 'channels.chan29.preset', 'channels.chan29.gate', 'channels.chan30', 'channels.chan30.chname', 'channels.chan30.preset', 'channels.chan30.gate', 'channels.chan31', 'channels.chan31.chname', 'channels.chan31.preset', 'channels.chan31.gate', 'channels.chan32', 'channels.chan32.chname', 'channels.chan32.preset', 'channels.chan32.gate', 'count_mode', 'delay', 'auto_count_delay', 'freq', 'preset_time', 'auto_count_time', 'egu'])"], 'num': 3}
	 num_intervals 2
	 plan_type generator
	 scan_id 1
	 plan_name count
	 num_points 3
	 detectors ['scaler']
	 hints {'dimensions': [(('time',), 'primary')]}
~~~~~~~~~~~~~~~~~
descriptor 8
	 run_start e2874f6b-7ea2-47eb-a000-80889341b212
	 time 1532552746.7983017
	 data_keys {'clock': {'source': 'PV:gov:scaler1.S1', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'I0': {'source': 'PV:gov:scaler1.S2', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'I': {'source': 'PV:gov:scaler1.S3', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scint': {'source': 'PV:gov:scaler1.S4', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'roi1': {'source': 'PV:gov:scaler1.S7', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scaler_time': {'source': 'PV:gov:scaler1.T', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}}
	 uid e7aad396-9ed1-43ac-9d8d-79efcb7c0678
	 configuration {'scaler': {'data': {'scaler_channels_chan01_chname': 'clock', 'scaler_channels_chan01_preset': 15000000.0, 'scaler_channels_chan01_gate': 'Y', 'scaler_channels_chan02_chname': 'I0', 'scaler_channels_chan02_preset': 0.0, 'scaler_channels_chan02_gate': 'N', 'scaler_channels_chan03_chname': 'I', 'scaler_channels_chan03_preset': 0.0, 'scaler_channels_chan03_gate': 'N', 'scaler_channels_chan04_chname': 'scint', 'scaler_channels_chan04_preset': 0.0, 'scaler_channels_chan04_gate': 'N', 'scaler_channels_chan05_chname': '', 'scaler_channels_chan05_preset': 0.0, 'scaler_channels_chan05_gate': 'N', 'scaler_channels_chan06_chname': '', 'scaler_channels_chan06_preset': 0.0, 'scaler_channels_chan06_gate': 'N', 'scaler_channels_chan07_chname': 'roi1', 'scaler_channels_chan07_preset': 0.0, 'scaler_channels_chan07_gate': 'N', 'scaler_channels_chan08_chname': '', 'scaler_channels_chan08_preset': 0.0, 'scaler_channels_chan08_gate': 'N', 'scaler_channels_chan09_chname': '', 'scaler_channels_chan09_preset': 0.0, 'scaler_channels_chan09_gate': 'N', 'scaler_channels_chan10_chname': '', 'scaler_channels_chan10_preset': 0.0, 'scaler_channels_chan10_gate': 'N', 'scaler_channels_chan11_chname': '', 'scaler_channels_chan11_preset': 0.0, 'scaler_channels_chan11_gate': 'N', 'scaler_channels_chan12_chname': '', 'scaler_channels_chan12_preset': 0.0, 'scaler_channels_chan12_gate': 'N', 'scaler_channels_chan13_chname': '', 'scaler_channels_chan13_preset': 0.0, 'scaler_channels_chan13_gate': 'N', 'scaler_channels_chan14_chname': '', 'scaler_channels_chan14_preset': 0.0, 'scaler_channels_chan14_gate': 'N', 'scaler_channels_chan15_chname': '', 'scaler_channels_chan15_preset': 0.0, 'scaler_channels_chan15_gate': 'N', 'scaler_channels_chan16_chname': '', 'scaler_channels_chan16_preset': 0.0, 'scaler_channels_chan16_gate': 'N', 'scaler_channels_chan17_chname': '', 'scaler_channels_chan17_preset': 0.0, 'scaler_channels_chan17_gate': 'N', 'scaler_channels_chan18_chname': '', 'scaler_channels_chan18_preset': 0.0, 'scaler_channels_chan18_gate': 'N', 'scaler_channels_chan19_chname': '', 'scaler_channels_chan19_preset': 0.0, 'scaler_channels_chan19_gate': 'N', 'scaler_channels_chan20_chname': '', 'scaler_channels_chan20_preset': 0.0, 'scaler_channels_chan20_gate': 'N', 'scaler_channels_chan21_chname': '', 'scaler_channels_chan21_preset': 0.0, 'scaler_channels_chan21_gate': 'N', 'scaler_channels_chan22_chname': '', 'scaler_channels_chan22_preset': 0.0, 'scaler_channels_chan22_gate': 'N', 'scaler_channels_chan23_chname': '', 'scaler_channels_chan23_preset': 0.0, 'scaler_channels_chan23_gate': 'N', 'scaler_channels_chan24_chname': '', 'scaler_channels_chan24_preset': 0.0, 'scaler_channels_chan24_gate': 'N', 'scaler_channels_chan25_chname': '', 'scaler_channels_chan25_preset': 0.0, 'scaler_channels_chan25_gate': 'N', 'scaler_channels_chan26_chname': '', 'scaler_channels_chan26_preset': 0.0, 'scaler_channels_chan26_gate': 'N', 'scaler_channels_chan27_chname': '', 'scaler_channels_chan27_preset': 0.0, 'scaler_channels_chan27_gate': 'N', 'scaler_channels_chan28_chname': '', 'scaler_channels_chan28_preset': 0.0, 'scaler_channels_chan28_gate': 'N', 'scaler_channels_chan29_chname': '', 'scaler_channels_chan29_preset': 0.0, 'scaler_channels_chan29_gate': 'N', 'scaler_channels_chan30_chname': '', 'scaler_channels_chan30_preset': 0.0, 'scaler_channels_chan30_gate': 'N', 'scaler_channels_chan31_chname': '', 'scaler_channels_chan31_preset': 0.0, 'scaler_channels_chan31_gate': 'N', 'scaler_channels_chan32_chname': '', 'scaler_channels_chan32_preset': 0.0, 'scaler_channels_chan32_gate': 'N', 'scaler_count_mode': 'OneShot', 'scaler_delay': 0.0, 'scaler_auto_count_delay': 1.0, 'scaler_freq': 10000000.0, 'scaler_preset_time': 1.5, 'scaler_auto_count_time': 1.0, 'scaler_egu': ''}, 'timestamps': {'scaler_channels_chan01_chname': 1532552746.617859, 'scaler_channels_chan01_preset': 1532552746.617859, 'scaler_channels_chan01_gate': 1532552746.617859, 'scaler_channels_chan02_chname': 1532552746.617859, 'scaler_channels_chan02_preset': 1532552746.617859, 'scaler_channels_chan02_gate': 1532552746.617859, 'scaler_channels_chan03_chname': 1532552746.617859, 'scaler_channels_chan03_preset': 1532552746.617859, 'scaler_channels_chan03_gate': 1532552746.617859, 'scaler_channels_chan04_chname': 1532552746.617859, 'scaler_channels_chan04_preset': 1532552746.617859, 'scaler_channels_chan04_gate': 1532552746.617859, 'scaler_channels_chan05_chname': 1532552746.617859, 'scaler_channels_chan05_preset': 1532552746.617859, 'scaler_channels_chan05_gate': 1532552746.617859, 'scaler_channels_chan06_chname': 1532552746.617859, 'scaler_channels_chan06_preset': 1532552746.617859, 'scaler_channels_chan06_gate': 1532552746.617859, 'scaler_channels_chan07_chname': 1532552746.617859, 'scaler_channels_chan07_preset': 1532552746.617859, 'scaler_channels_chan07_gate': 1532552746.617859, 'scaler_channels_chan08_chname': 1532552746.617859, 'scaler_channels_chan08_preset': 1532552746.617859, 'scaler_channels_chan08_gate': 1532552746.617859, 'scaler_channels_chan09_chname': 1532552746.617859, 'scaler_channels_chan09_preset': 1532552746.617859, 'scaler_channels_chan09_gate': 1532552746.617859, 'scaler_channels_chan10_chname': 1532552746.617859, 'scaler_channels_chan10_preset': 1532552746.617859, 'scaler_channels_chan10_gate': 1532552746.617859, 'scaler_channels_chan11_chname': 1532552746.617859, 'scaler_channels_chan11_preset': 1532552746.617859, 'scaler_channels_chan11_gate': 1532552746.617859, 'scaler_channels_chan12_chname': 1532552746.617859, 'scaler_channels_chan12_preset': 1532552746.617859, 'scaler_channels_chan12_gate': 1532552746.617859, 'scaler_channels_chan13_chname': 1532552746.617859, 'scaler_channels_chan13_preset': 1532552746.617859, 'scaler_channels_chan13_gate': 1532552746.617859, 'scaler_channels_chan14_chname': 1532552746.617859, 'scaler_channels_chan14_preset': 1532552746.617859, 'scaler_channels_chan14_gate': 1532552746.617859, 'scaler_channels_chan15_chname': 1532552746.617859, 'scaler_channels_chan15_preset': 1532552746.617859, 'scaler_channels_chan15_gate': 1532552746.617859, 'scaler_channels_chan16_chname': 1532552746.617859, 'scaler_channels_chan16_preset': 1532552746.617859, 'scaler_channels_chan16_gate': 1532552746.617859, 'scaler_channels_chan17_chname': 1532552746.617859, 'scaler_channels_chan17_preset': 1532552746.617859, 'scaler_channels_chan17_gate': 1532552746.617859, 'scaler_channels_chan18_chname': 1532552746.617859, 'scaler_channels_chan18_preset': 1532552746.617859, 'scaler_channels_chan18_gate': 1532552746.617859, 'scaler_channels_chan19_chname': 1532552746.617859, 'scaler_channels_chan19_preset': 1532552746.617859, 'scaler_channels_chan19_gate': 1532552746.617859, 'scaler_channels_chan20_chname': 1532552746.617859, 'scaler_channels_chan20_preset': 1532552746.617859, 'scaler_channels_chan20_gate': 1532552746.617859, 'scaler_channels_chan21_chname': 1532552746.617859, 'scaler_channels_chan21_preset': 1532552746.617859, 'scaler_channels_chan21_gate': 1532552746.617859, 'scaler_channels_chan22_chname': 1532552746.617859, 'scaler_channels_chan22_preset': 1532552746.617859, 'scaler_channels_chan22_gate': 1532552746.617859, 'scaler_channels_chan23_chname': 1532552746.617859, 'scaler_channels_chan23_preset': 1532552746.617859, 'scaler_channels_chan23_gate': 1532552746.617859, 'scaler_channels_chan24_chname': 1532552746.617859, 'scaler_channels_chan24_preset': 1532552746.617859, 'scaler_channels_chan24_gate': 1532552746.617859, 'scaler_channels_chan25_chname': 1532552746.617859, 'scaler_channels_chan25_preset': 1532552746.617859, 'scaler_channels_chan25_gate': 1532552746.617859, 'scaler_channels_chan26_chname': 1532552746.617859, 'scaler_channels_chan26_preset': 1532552746.617859, 'scaler_channels_chan26_gate': 1532552746.617859, 'scaler_channels_chan27_chname': 1532552746.617859, 'scaler_channels_chan27_preset': 1532552746.617859, 'scaler_channels_chan27_gate': 1532552746.617859, 'scaler_channels_chan28_chname': 1532552746.617859, 'scaler_channels_chan28_preset': 1532552746.617859, 'scaler_channels_chan28_gate': 1532552746.617859, 'scaler_channels_chan29_chname': 1532552746.617859, 'scaler_channels_chan29_preset': 1532552746.617859, 'scaler_channels_chan29_gate': 1532552746.617859, 'scaler_channels_chan30_chname': 1532552746.617859, 'scaler_channels_chan30_preset': 1532552746.617859, 'scaler_channels_chan30_gate': 1532552746.617859, 'scaler_channels_chan31_chname': 1532552746.617859, 'scaler_channels_chan31_preset': 1532552746.617859, 'scaler_channels_chan31_gate': 1532552746.617859, 'scaler_channels_chan32_chname': 1532552746.617859, 'scaler_channels_chan32_preset': 1532552746.617859, 'scaler_channels_chan32_gate': 1532552746.617859, 'scaler_count_mode': 1532552746.617859, 'scaler_delay': 1532552746.617859, 'scaler_auto_count_delay': 1532552746.617859, 'scaler_freq': 1532552746.617859, 'scaler_preset_time': 1532552746.617859, 'scaler_auto_count_time': 1532552746.617859, 'scaler_egu': 1532552746.617859}, 'data_keys': OrderedDict([('scaler_channels_chan01_chname', {'source': 'PV:gov:scaler1.NM1', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan01_preset', {'source': 'PV:gov:scaler1.PR1', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan01_gate', {'source': 'PV:gov:scaler1.G1', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan02_chname', {'source': 'PV:gov:scaler1.NM2', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan02_preset', {'source': 'PV:gov:scaler1.PR2', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan02_gate', {'source': 'PV:gov:scaler1.G2', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan03_chname', {'source': 'PV:gov:scaler1.NM3', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan03_preset', {'source': 'PV:gov:scaler1.PR3', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan03_gate', {'source': 'PV:gov:scaler1.G3', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan04_chname', {'source': 'PV:gov:scaler1.NM4', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan04_preset', {'source': 'PV:gov:scaler1.PR4', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan04_gate', {'source': 'PV:gov:scaler1.G4', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan05_chname', {'source': 'PV:gov:scaler1.NM5', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan05_preset', {'source': 'PV:gov:scaler1.PR5', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan05_gate', {'source': 'PV:gov:scaler1.G5', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan06_chname', {'source': 'PV:gov:scaler1.NM6', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan06_preset', {'source': 'PV:gov:scaler1.PR6', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan06_gate', {'source': 'PV:gov:scaler1.G6', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan07_chname', {'source': 'PV:gov:scaler1.NM7', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan07_preset', {'source': 'PV:gov:scaler1.PR7', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan07_gate', {'source': 'PV:gov:scaler1.G7', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan08_chname', {'source': 'PV:gov:scaler1.NM8', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan08_preset', {'source': 'PV:gov:scaler1.PR8', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan08_gate', {'source': 'PV:gov:scaler1.G8', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan09_chname', {'source': 'PV:gov:scaler1.NM9', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan09_preset', {'source': 'PV:gov:scaler1.PR9', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan09_gate', {'source': 'PV:gov:scaler1.G9', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan10_chname', {'source': 'PV:gov:scaler1.NM10', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan10_preset', {'source': 'PV:gov:scaler1.PR10', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan10_gate', {'source': 'PV:gov:scaler1.G10', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan11_chname', {'source': 'PV:gov:scaler1.NM11', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan11_preset', {'source': 'PV:gov:scaler1.PR11', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan11_gate', {'source': 'PV:gov:scaler1.G11', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan12_chname', {'source': 'PV:gov:scaler1.NM12', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan12_preset', {'source': 'PV:gov:scaler1.PR12', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan12_gate', {'source': 'PV:gov:scaler1.G12', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan13_chname', {'source': 'PV:gov:scaler1.NM13', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan13_preset', {'source': 'PV:gov:scaler1.PR13', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan13_gate', {'source': 'PV:gov:scaler1.G13', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan14_chname', {'source': 'PV:gov:scaler1.NM14', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan14_preset', {'source': 'PV:gov:scaler1.PR14', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan14_gate', {'source': 'PV:gov:scaler1.G14', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan15_chname', {'source': 'PV:gov:scaler1.NM15', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan15_preset', {'source': 'PV:gov:scaler1.PR15', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan15_gate', {'source': 'PV:gov:scaler1.G15', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan16_chname', {'source': 'PV:gov:scaler1.NM16', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan16_preset', {'source': 'PV:gov:scaler1.PR16', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan16_gate', {'source': 'PV:gov:scaler1.G16', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan17_chname', {'source': 'PV:gov:scaler1.NM17', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan17_preset', {'source': 'PV:gov:scaler1.PR17', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan17_gate', {'source': 'PV:gov:scaler1.G17', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan18_chname', {'source': 'PV:gov:scaler1.NM18', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan18_preset', {'source': 'PV:gov:scaler1.PR18', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan18_gate', {'source': 'PV:gov:scaler1.G18', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan19_chname', {'source': 'PV:gov:scaler1.NM19', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan19_preset', {'source': 'PV:gov:scaler1.PR19', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan19_gate', {'source': 'PV:gov:scaler1.G19', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan20_chname', {'source': 'PV:gov:scaler1.NM20', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan20_preset', {'source': 'PV:gov:scaler1.PR20', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan20_gate', {'source': 'PV:gov:scaler1.G20', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan21_chname', {'source': 'PV:gov:scaler1.NM21', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan21_preset', {'source': 'PV:gov:scaler1.PR21', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan21_gate', {'source': 'PV:gov:scaler1.G21', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan22_chname', {'source': 'PV:gov:scaler1.NM22', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan22_preset', {'source': 'PV:gov:scaler1.PR22', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan22_gate', {'source': 'PV:gov:scaler1.G22', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan23_chname', {'source': 'PV:gov:scaler1.NM23', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan23_preset', {'source': 'PV:gov:scaler1.PR23', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan23_gate', {'source': 'PV:gov:scaler1.G23', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan24_chname', {'source': 'PV:gov:scaler1.NM24', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan24_preset', {'source': 'PV:gov:scaler1.PR24', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan24_gate', {'source': 'PV:gov:scaler1.G24', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan25_chname', {'source': 'PV:gov:scaler1.NM25', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan25_preset', {'source': 'PV:gov:scaler1.PR25', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan25_gate', {'source': 'PV:gov:scaler1.G25', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan26_chname', {'source': 'PV:gov:scaler1.NM26', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan26_preset', {'source': 'PV:gov:scaler1.PR26', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan26_gate', {'source': 'PV:gov:scaler1.G26', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan27_chname', {'source': 'PV:gov:scaler1.NM27', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan27_preset', {'source': 'PV:gov:scaler1.PR27', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan27_gate', {'source': 'PV:gov:scaler1.G27', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan28_chname', {'source': 'PV:gov:scaler1.NM28', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan28_preset', {'source': 'PV:gov:scaler1.PR28', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan28_gate', {'source': 'PV:gov:scaler1.G28', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan29_chname', {'source': 'PV:gov:scaler1.NM29', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan29_preset', {'source': 'PV:gov:scaler1.PR29', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan29_gate', {'source': 'PV:gov:scaler1.G29', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan30_chname', {'source': 'PV:gov:scaler1.NM30', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan30_preset', {'source': 'PV:gov:scaler1.PR30', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan30_gate', {'source': 'PV:gov:scaler1.G30', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan31_chname', {'source': 'PV:gov:scaler1.NM31', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan31_preset', {'source': 'PV:gov:scaler1.PR31', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan31_gate', {'source': 'PV:gov:scaler1.G31', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_channels_chan32_chname', {'source': 'PV:gov:scaler1.NM32', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_channels_chan32_preset', {'source': 'PV:gov:scaler1.PR32', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_channels_chan32_gate', {'source': 'PV:gov:scaler1.G32', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['N', 'Y']}), ('scaler_count_mode', {'source': 'PV:gov:scaler1.CONT', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['OneShot', 'AutoCount']}), ('scaler_delay', {'source': 'PV:gov:scaler1.DLY', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_auto_count_delay', {'source': 'PV:gov:scaler1.DLY1', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_freq', {'source': 'PV:gov:scaler1.FREQ', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_preset_time', {'source': 'PV:gov:scaler1.TP', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_auto_count_time', {'source': 'PV:gov:scaler1.TP1', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_egu', {'source': 'PV:gov:scaler1.EGU', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None})])}}
	 name primary
	 hints {'scaler': {'fields': ['clock', 'I0', 'I', 'scint', 'roi1']}}
	 object_keys {'scaler': ['clock', 'I0', 'I', 'scint', 'roi1', 'scaler_time']}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor e7aad396-9ed1-43ac-9d8d-79efcb7c0678
	 time 1532552746.809361
	 data {'clock': 16000000.0, 'I0': 9.0, 'I': 5.0, 'scint': 8.0, 'roi1': 9.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1532552746.617859, 'I0': 1532552746.617859, 'I': 1532552746.617859, 'scint': 1532552746.617859, 'roi1': 1532552746.617859, 'scaler_time': 1532552746.617859}
	 seq_num 1
	 uid d827abc5-c521-4cbe-b2c3-b8fc1486f3e8
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor e7aad396-9ed1-43ac-9d8d-79efcb7c0678
	 time 1532552748.530158
	 data {'clock': 16000000.0, 'I0': 8.0, 'I': 10.0, 'scint': 7.0, 'roi1': 7.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1532552748.524776, 'I0': 1532552748.524776, 'I': 1532552748.524776, 'scint': 1532552748.524776, 'roi1': 1532552748.524776, 'scaler_time': 1532552748.524776}
	 seq_num 2
	 uid 0867de60-789d-459b-bc1d-471911ab60f6
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor e7aad396-9ed1-43ac-9d8d-79efcb7c0678
	 time 1532552750.15568
	 data {'clock': 16000000.0, 'I0': 6.0, 'I': 7.0, 'scint': 7.0, 'roi1': 6.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1532552750.150283, 'I0': 1532552750.150283, 'I': 1532552750.150283, 'scint': 1532552750.150283, 'roi1': 1532552750.150283, 'scaler_time': 1532552750.150283}
	 seq_num 3
	 uid 66335b8d-fbf4-4e3e-85d2-1f14cd30b8df
	 filled {}
~~~~~~~~~~~~~~~~~
stop 6
	 run_start e2874f6b-7ea2-47eb-a000-80889341b212
	 time 1532552750.1573653
	 uid cf5a1791-280c-43f2-8ab7-320096dc61ef
	 exit_status success
	 reason 
	 num_events {'primary': 3}
~~~~~~~~~~~~~~~~~
('e2874f6b-7ea2-47eb-a000-80889341b212',)
```


## Quit the jupyter notebook

* Save the "lesson1" page if you wish.
* Press the **Logout** button in the upper right corner of "lesson1" and close the page.
* Press the **Logout** button in the upper right corner of "Home" and close the page.
* Type Control-C (^C) twice in the jupyter-notebook console to kill the server.